In [67]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df=pd.read_csv('/home/mllab/Downloads/NBC/NBC/Iris.csv')
mydata=df.values.tolist() #or np.array(df)

In [55]:
#convert the last string (iris-sentosa) label to numberical labels

def classtolabel(mydata):
    classes=[]
    for i in range(len(mydata)):
        if mydata[i][-1] not in classes:
            classes.append(mydata[i][-1])
    for i in range(len(classes)):
        for j in range(len(mydata)):
            if mydata[j][-1]==classes[i]:
                mydata[j][-1]=i
    return mydata
classtolabel(mydata)
     

[[1, 5.1, 3.5, 1.4, 0.2, 0],
 [2, 4.9, 3.0, 1.4, 0.2, 0],
 [3, 4.7, 3.2, 1.3, 0.2, 0],
 [4, 4.6, 3.1, 1.5, 0.2, 0],
 [5, 5.0, 3.6, 1.4, 0.2, 0],
 [6, 5.4, 3.9, 1.7, 0.4, 0],
 [7, 4.6, 3.4, 1.4, 0.3, 0],
 [8, 5.0, 3.4, 1.5, 0.2, 0],
 [9, 4.4, 2.9, 1.4, 0.2, 0],
 [10, 4.9, 3.1, 1.5, 0.1, 0],
 [11, 5.4, 3.7, 1.5, 0.2, 0],
 [12, 4.8, 3.4, 1.6, 0.2, 0],
 [13, 4.8, 3.0, 1.4, 0.1, 0],
 [14, 4.3, 3.0, 1.1, 0.1, 0],
 [15, 5.8, 4.0, 1.2, 0.2, 0],
 [16, 5.7, 4.4, 1.5, 0.4, 0],
 [17, 5.4, 3.9, 1.3, 0.4, 0],
 [18, 5.1, 3.5, 1.4, 0.3, 0],
 [19, 5.7, 3.8, 1.7, 0.3, 0],
 [20, 5.1, 3.8, 1.5, 0.3, 0],
 [21, 5.4, 3.4, 1.7, 0.2, 0],
 [22, 5.1, 3.7, 1.5, 0.4, 0],
 [23, 4.6, 3.6, 1.0, 0.2, 0],
 [24, 5.1, 3.3, 1.7, 0.5, 0],
 [25, 4.8, 3.4, 1.9, 0.2, 0],
 [26, 5.0, 3.0, 1.6, 0.2, 0],
 [27, 5.0, 3.4, 1.6, 0.4, 0],
 [28, 5.2, 3.5, 1.5, 0.2, 0],
 [29, 5.2, 3.4, 1.4, 0.2, 0],
 [30, 4.7, 3.2, 1.6, 0.2, 0],
 [31, 4.8, 3.1, 1.6, 0.2, 0],
 [32, 5.4, 3.4, 1.5, 0.4, 0],
 [33, 5.2, 4.1, 1.5, 0.1, 0],
 [34, 5.5, 4.2, 1.4

In [38]:
#splits into training and testing data set with 80% training and 100% testing

def splitting(mydata, ratio):
    train_num = int(len(mydata) * ratio)
    train = []
    test = list(mydata)
    while len(train) < train_num:
        index = random.randrange(len(test))
        train.append(test.pop(index))
    return train, test
train_data, test_data = splitting(mydata, 0.8)

In [47]:
#makes dictionary and groups according to the class

def groupUnderClass(mydata):
    data_dict = {}
    for i in range(len(mydata)):
        if mydata[i][-1] not in data_dict:
            data_dict[mydata[i][-1]] = []
        data_dict[mydata[i][-1]].append(mydata[i])
    return data_dict
#groupUnderClass(mydata)

In [56]:
#calculate mean and standard deviation form {classlabel:[mean,std],[]...}

def MeanAndStdDev(numbers):
    avg = np.mean(numbers)
    stddev = np.std(numbers)
    return avg, stddev

def MeanAndStdDevForClass(mydata):
    info = {}
    data_dict = groupUnderClass(mydata)
    #print(data_dict)
    for classValue, instances in data_dict.items():
        info[classValue] = [MeanAndStdDev(attribute) for attribute in zip(*instances)]
    return info
MeanAndStdDevForClass(mydata)


{0: [(25.5, 14.430869689661812),
  (5.006, 0.3489469873777391),
  (3.418, 0.37719490982779713),
  (1.464, 0.17176728442867112),
  (0.244, 0.10613199329137281),
  (0.0, 0.0)],
 1: [(75.5, 14.430869689661812),
  (5.936, 0.5109833656783751),
  (2.7700000000000005, 0.31064449134018135),
  (4.26, 0.4651881339845203),
  (1.3259999999999998, 0.19576516544063705),
  (1.0, 0.0)],
 2: [(125.5, 14.430869689661812),
  (6.587999999999998, 0.6294886813914926),
  (2.974, 0.3192553836664309),
  (5.5520000000000005, 0.546347874526844),
  (2.0260000000000002, 0.2718896835115301),
  (2.0, 0.0)]}

In [66]:
#Calculating Gaussian probability
def calculateGauss(x, mean, stdev):
    epsilon = 1e-10 #epsilon used so as to avoid division by zero error
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev + epsilon, 2))))
    return (1 / (math.sqrt(2 * math.pi) * (stdev + epsilon))) * expo

def calculateClassProbability(info, eachtest):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = eachtest[i] #checks for each column in that test row
            probabilities[classValue] *= calculateGauss(x, mean, std_dev)
    return probabilities

def predict(info, test):
    probabilities = calculateClassProbability(info, test)
    bestLabel = max(probabilities, key=probabilities.get)
    return bestLabel

def getPredictions(info, test):
    predictions = [predict(info, instance) for instance in test]
    return predictions

def accuracy_rate(test, predictions):
    correct = sum(1 for i in range(len(test)) if test[i][-1] == predictions[i])
    return (correct / float(len(test))) * 100.0

ratio = 0.8
train_data, test_data = splitting(mydata, ratio)

print('Total number of examples:', len(mydata))
print('Training examples:', len(train_data))
print('Test examples:', len(test_data))

info = MeanAndStdDevForClass(train_data)

# Test the model
predictions = getPredictions(info, test_data)
accuracy = accuracy_rate(test_data, predictions)
print('Accuracy of the model:', accuracy)

Total number of examples: 150
Training examples: 120
Test examples: 30
Accuracy of the model: 100.0
